In [2]:
import joblib

preprocessor = joblib.load("../models/preprocessor.pkl")
X_train_proc, X_test_proc, y_train, y_test = joblib.load("../models/splits.pkl")

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    "penalty": ["l1", "l2"],
    "C": [0.01, 0.1, 1, 10],
    "solver": ["liblinear"]
}

grid = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring="accuracy", n_jobs=-1)
grid.fit(X_train_proc, y_train)

print("Best Parameters (Logistic Regression):", grid.best_params_)
print("Best CV Score:", grid.best_score_)

best_log_reg = grid.best_estimator_

Best Parameters (Logistic Regression): {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best CV Score: 0.8345238095238097


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [None, 10, 20, 30, 40],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False]
}

random_search = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1
)
random_search.fit(X_train_proc, y_train)

print("Best Parameters (Random Forest):", random_search.best_params_)
print("Best CV Score:", random_search.best_score_)

best_rf = random_search.best_estimator_

Best Parameters (Random Forest): {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_depth': 10, 'bootstrap': False}
Best CV Score: 0.8385204081632655


In [6]:
from sklearn.metrics import accuracy_score, f1_score, classification_report

for name, model in [("Best Logistic Regression", best_log_reg), ("Best Random Forest", best_rf)]:
    y_pred = model.predict(X_test_proc)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average="weighted")
    print(f"{name} - Accuracy: {acc:.3f}, F1: {f1:.3f}")
    print(classification_report(y_test, y_pred))
    print("="*50)

Best Logistic Regression - Accuracy: 0.869, F1: 0.869
              precision    recall  f1-score   support

           0       0.93      0.82      0.87        33
           1       0.81      0.93      0.87        28

    accuracy                           0.87        61
   macro avg       0.87      0.87      0.87        61
weighted avg       0.88      0.87      0.87        61

Best Random Forest - Accuracy: 0.885, F1: 0.885
              precision    recall  f1-score   support

           0       0.93      0.85      0.89        33
           1       0.84      0.93      0.88        28

    accuracy                           0.89        61
   macro avg       0.89      0.89      0.89        61
weighted avg       0.89      0.89      0.89        61



In [7]:
import joblib
import os

os.makedirs("../models", exist_ok=True)

joblib.dump(best_log_reg, "../models/best_logistic_reg.pkl")

joblib.dump(best_rf, "../models/best_random_forest.pkl")

print("✅ Models saved successfully in ../models/")

✅ Models saved successfully in ../models/
